**student**: Titouan Le Breton

In [1]:
import numpy as np

# Exercise 1.A

### Question 1

A gibbs sampler is a sampling one cordinate conditionaly on the others. Sometimes, you can't sample this conditional, so you need MH.

It seems we select one o fthe indexes at random.


Here, we use MH to sample tehe condtional distribution pi(x_i|x_-i)

so the algirthm is: 

for step in n:
    sample i form 1,2
    update x_i with MH

P_1 and P_2 sample the first and second coordinate respectively. However, they take as input z in R2.... which will have one of this coordinates unchanged

we are in the case where pi(x_prop|y) / pi(x|y) = pi(x_prop, y) / pi(x, y)

### Question 2 and 3

In [ ]:
def pi(x,y , a=10):
    store_var = -np.power(x,2) / a**2 - np.power(y,2) - 0.25*np.power(np.power(x,2)/a**2 - np.power(y,2) ,2)
    return np.exp(store_var)

x = np.random.randn(5)
y = np.random.randn(5)

array([0.15178705, 0.96078197, 0.8429807 , 0.77700862, 0.567476  ])

In [37]:
def sample_P1(x, y, sigma_1, pi):
    x_prop = x + np.random.normal(0, sigma_1)
    alpha = min(1, pi(x_prop, y) / pi(x, y))

    if np.random.rand() < alpha:
        return x_prop, y
    else:
        return x, y


def sample_P2(x, y, sigma_2, pi):
    y_prop = y + np.random.normal(0, sigma_2)
    alpha = min(1, pi(x, y_prop) / pi(x, y))

    if np.random.rand() < alpha:
        return x, y_prop
    else:
        return x, y

In [48]:
def MH_with_gibs(sigma_1, sigma_2, pi, N_steps, return_acceptance_ratio = False):
    #just run the sampler N_steps times
    x = np.random.randn()
    y = np.random.randn()
    count_x_accepted = 0
    count_y_accepted = 0
    x_list = [x]
    y_list = [y]
    for i in range(N_steps):
        x_new,y = sample_P1(x, y, sigma_1, pi)
        x_new,y_new = sample_P2(x_new, y, sigma_2, pi)

        if x_new != x_new:
            count_x_accepted += 1
        if y_new != y:
            count_y_accepted += 1
            
        x, y = x_new, y_new
        x_list.append(x)
        y_list.append(y)
    if return_acceptance_ratio:
        return x_list, y_list, count_x_accepted/ N_steps, count_y_accepted / N_steps
    else:
        return x_list, y_list
        

sigma_1 = 3
sigma_2 = 3
N_steps = 50

x_MH, y_MH = MH_with_gibs(sigma_1, sigma_2, pi, N_steps, return_acceptance_ratio = False)     


In [41]:
len(x_list)

51

### Q4

We do not get good results. We move a lot on x, but not a lot on y. So we accept a lot on x but not on y (we get horiztional lines). It is an issue of variance: it is too small on the x (we accept to often) and to big on y (the proposition are too drasticall and we reject too often)

Solution? make sigma_1 larger and sigma_2 smaller


To check the quality of the markov chain, you can do an autocorrelation plot (you want it to go to 0 when the lag is large so you get indepndence)

# Exercise 1.B

The idea is to modify the variances to always have an acceptable acceptance rate (around 0.25%)

### Question 1

same as before, but after 50 steps we change the variance

Note: in practice, we look at effective acceptance ratio (not the average alphas)

In [49]:
def update_variance(sigma_1, sigma_2, x_acceptance_ratio, y_acceptance_ratio,sigma_j, golden_ratio):
    if x_acceptance_ratio > golden_ratio:
        sigma_1 -= sigma_j
    else:
        sigma_1 += sigma_j 
    
    if y_acceptance_ratio > golden_ratio:
        sigma_2 -= sigma_j
    else:
        sigma_2 += sigma_j 
    
    return sigma_1, sigma_2


def adaptive_MH_with_gibs(sigma_1, sigma_2, pi, N_batches):
    x_list = []
    y_list = []
    N_steps = 50
    golden_ratio = 0.234
    for j in range(1,N_batches+1):
        # do the sampling N_steps times
        x, y, x_acceptance_ratio, y_acceptance_ratio = MH_with_gibs(sigma_1, sigma_2, pi, N_steps, return_acceptance_ratio = True)
        x_list.append(x)
        y_list.append(y)

        # handle the modified vaiances to get a better acceptance ratio
        sigma_j = min(0.05, j**(-0.5))
        sigma_1, sigma_2 = update_variance(sigma_1, sigma_2, x_acceptance_ratio, y_acceptance_ratio,sigma_j, golden_ratio)
    
    return x_list, y_list

N_batches = 5
x_adaptive_MH, y_adaptive_MH = adaptive_MH_with_gibs(sigma_1, sigma_2, pi, N_batches)

# Exercise 2.A

### Question 1

In [50]:
mu = np.array([
    [2.18, 5.76], [8.67, 9.59], [4.24, 8.48], [8.41, 1.68], [3.93, 8.82], [3.25, 3.47], [1.70, 0.50], [4.59, 5.60], [6.91, 5.81], [6.87, 5.40],[5.41, 2.65],
    [2.70, 7.88],[4.98, 3.70],[1.14, 2.39],[8.33, 9.50],[4.93, 1.50],[1.83, 0.09],[2.26, 0.31],[5.54, 6.86],[1.69, 8.11]
])

In [57]:
mu.shape

(20, 2)

In [60]:
sigma = 0.1
w = 0.05

x = np.array([1,-2])
X = np.tile(x, (20, 1))


exp_matrix = np.exp( (X - mu) @ (X - mu).T / (-2 * sigma**2))
exp_matrix.shape

(20, 20)

# Exercise 2.B

### Question 1